In [ ]:
        # ================== Imports & Config ==================
import asyncio, random, re, time
from pathlib import Path
from datetime import datetime
from playwright.async_api import async_playwright
import pandas as pd
from bs4 import BeautifulSoup

LIST_URL = "https://business.columbia.edu/faculty/research?viewsreference%5Bcompressed%5D=eJxdkNEKgzAMRf8lz32YCnP4MyXSqIG2Sq2OIf770hXXsYeShHNvbugBBiNCdwCGcXPkI3R-s1bBRGgoXJNlx4KgqkHBPAwrfYULjkkHg0wCI0dLmZ0KyGNvyWjRR_bjmpIuQ67qJ7m0JTHXM-UEIVoex5eOr4U-OwKOAZcJ_gVsBNdN0xQyMFmjPbrkzEPy653puWpyvRya-uIItPPKs8_b2vZeVQ-h2cuRnDZk0__dzjeCjnKj&page=1"

BASE = "https://business.columbia.edu"
PAGES_DIR = Path("columbia_pages")
PROFILE_DIR = Path(r"C:\temp\columbia_chrome_profile").absolute()
HEADLESS = False
USER_AGENT = ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
              "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0 Safari/537.36")
PAGES_DIR.mkdir(exist_ok=True, parents=True)

# ================== Helpers ==================
def set_page(url: str, n: int) -> str:
    if re.search(r"([?&])page=\d+", url):
        return re.sub(r"([?&])page=\d+", rf"\1page={n}", url)
    sep = "&" if "?" in url else "?"
    return f"{url}{sep}page={n}"

def clean(s: str) -> str:
    if not s: return ""
    s = s.replace("\xa0", " ").replace("\u200b", " ")
    return re.sub(r"\s+", " ", s).strip()

async def pass_human_gate(page):
    print("⏳ Manual verification may be required... please click 'I Agree / I'm not a robot' in the browser")
    for _ in range(60):
        title = (await page.title()) or ""
        html = (await page.content()) or ""
        if "business.columbia.edu" in page.url and not re.search(r"verify|checking|robot", title+html, re.I):
            print("✅ Verification passed, continuing")
            return
        await asyncio.sleep(3)
    print("⚠️ Timeout: manual verification may not have been completed")

async def get_readmore_links(page):
    await page.wait_for_selector("article.research-archive-item, li.m-listing-item", timeout=60_000)
    sel = (
        "article.research-archive-item .m-listing-item_actions a.a-link-strong--secondary, "
        "li.m-listing-item .m-listing-item_actions a.a-link-strong--secondary, "
        "a:has-text('Read more'), a:has-text('Read More')"
    )
    nodes = await page.locator(sel).all()
    urls = []
    for n in nodes:
        href = await n.get_attribute("href")
        if href and "/faculty/research/" in href:
            urls.append(href if href.startswith("http") else BASE + href)
    seen, out = set(), []
    for u in urls:
        if u not in seen:
            seen.add(u)
            out.append(u)
    return out

async def fetch_detail_with_retry(page, url, tries=4):
    base = url
    for i in range(1, tries + 1):
        try:
            bust = f"{int(time.time()*1000)}-{i}"
            u = base + ("&" if "?" in base else "?") + f"_cb={bust}"
            await page.goto(u, wait_until="domcontentloaded", timeout=180_000)
            html = await page.content()
            if re.search(r"verify|checking|robot", html.lower()):
                await pass_human_gate(page)
            for _ in range(10):
                await page.mouse.wheel(0, 1400); await asyncio.sleep(0.15)
            await page.wait_for_selector("article.node, .node-citation--full-citation, dl.citation-meta--item, .citation", timeout=120_000)
            return await page.content()
        except Exception:
            await asyncio.sleep(0.8 + i * 0.7)
    return None

def parse_detail_html(html: str, url: str) -> dict:
    soup = BeautifulSoup(html, "html.parser")
    citation_as_title = ""
    cit = soup.select_one(".node-citation--full-citation, .node-citation--full, .article-citation, .citation")
    if cit:
        p = cit.select_one("p")
        citation_as_title = clean(p.get_text(" ", strip=True) if p else cit.get_text(" ", strip=True))
    if not citation_as_title:
        h1 = soup.select_one("h1")
        citation_as_title = clean(h1.get_text(" ", strip=True)) if h1 else ""
    citation_as_title = re.sub(r"^Full Citation\s*", "", citation_as_title, flags=re.I)

    authors, journal, year = "", "", ""
    for block in soup.select("dl.citation-meta--item, dl.citation-meta__item, dl.citation-meta, dl.citation"):
        dt = block.select_one("dt"); dd = block.select_one("dd")
        if not dt or not dd: continue
        key = clean(dt.get_text()).lower()
        if "author" in key:
            parts = [clean(x.get_text()) for x in dd.select("a, .field_item, .field__item") if clean(x.get_text())]
            authors = ", ".join(parts) if parts else clean(dd.get_text())
        elif "journal" in key:
            journal = clean(dd.get_text())
        elif "date" in key:
            m = re.search(r"\d{4}", dd.get_text())
            if m: year = m.group()

    abstract = ""
    h2_abs = None
    for h2 in soup.select("h2"):
        if clean(h2.get_text()).lower() == "abstract":
            h2_abs = h2; break
    if h2_abs:
        paras = []
        for sib in h2_abs.next_siblings:
            if getattr(sib, "name", None) in {"h2", "dl"}:
                break
            for p in BeautifulSoup(str(sib), "html.parser").select("p"):
                txt = clean(p.get_text())
                if txt: paras.append(txt)
        abstract = " ".join(paras)
    if not abstract:
        candidates = soup.select('[class*="field"][class*="body"] p')
        abstract = " ".join([clean(p.get_text()) for p in candidates if clean(p.get_text())])
    if not abstract:
        p0 = soup.select_one("article.node p")
        abstract = clean(p0.get_text()) if p0 else ""
    return {
        "citation_as_title": citation_as_title,
        "authors": authors,
        "abstract": abstract,
        "journal": journal,
        "year": year,
        "url": url
    }

# ================== Main (merge into one Excel) ==================
async def main_many(start_page: int = 12, end_page: int = 21):
    all_records = []
    async with async_playwright() as pw:
        ctx = await pw.chromium.launch_persistent_context(
            channel="chrome",
            user_data_dir=str(PROFILE_DIR),
            headless=HEADLESS,
            viewport={"width": 1400, "height": 1000},
            args=["--lang=en-US,en","--disable-blink-features=AutomationControlled","--disable-gpu","--no-sandbox"]
        )
        page = await ctx.new_page()
        await page.set_extra_http_headers({"User-Agent": USER_AGENT})
        first_url = set_page(LIST_URL, start_page)
        print("👉 Please complete human verification in the browser (e.g., Cloudflare checks)")
        await page.goto(first_url, wait_until="domcontentloaded", timeout=180_000)
        await pass_human_gate(page)

        for p in range(start_page, end_page + 1):
            list_url = set_page(LIST_URL, p)
            print(f"\n===== 📄 Scraping list page page={p} =====")
            await page.goto(list_url, wait_until="domcontentloaded", timeout=180_000)
            for _ in range(8):
                await page.mouse.wheel(0, 1200)
                await asyncio.sleep(0.25)
            links = await get_readmore_links(page)
            print(f"🔗 Found {len(links)} links on this page")

            for i, link in enumerate(links, 1):
                html = await fetch_detail_with_retry(page, link)
                if not html:
                    print(f"⚠️ Skipped: {link}")
                    continue
                rec = parse_detail_html(html, link)
                all_records.append(rec)
                print(f"  ✅ {i}/{len(links)} {rec['citation_as_title'][:60]}...")
                await asyncio.sleep(random.uniform(0.8, 1.5))

        await ctx.close()

    df = pd.DataFrame(all_records)
    combined_path = PAGES_DIR / f"columbia_merged_{start_page}-{end_page}_{datetime.now():%Y%m%d_%H%M%S}.xlsx"
    df.to_excel(combined_path, index=False)
    print(f"\n💾 Merged {len(df)} records → {combined_path}")

# ================== Run ==================
import nest_asyncio
nest_asyncio.apply()
await main_many(0, 85)#change the page range as you needed


In [5]:
import pandas as pd
df = pd.read_excel("columbia_publications.xlsx")
print("\nTotal rows:", len(df))
df.head(20)




Total rows: 900


,citation_as_title,authors,abstract,journal,year,url
0,"Piper, Andrew and Olivier Toubia . “A Quantita...","Andrew Piper, and Olivier Toubia, Olivier Toubia",NaN,Poetics,2023.0,https://business.columbia.edu/faculty/research...
1,"Fan, Tingting , Peter N. Golder , and Donald L...","Tingting Fan, , Peter N. Golder, , and Donald ...",NaN,Models for Managerial Decision-Making,2023.0,https://business.columbia.edu/faculty/research...
2,"Hong, Harrison , Jinqiang Yang , and Neng Wang...","Harrison Hong, , Jinqiang Yang, , and Neng Wan...",Emissions abatement alone cannot address the c...,NaN,NaN,https://business.columbia.edu/faculty/research...
3,"Barahona, Nano , Cristobal Otero Ruiz-Tagle , ...","Nano Barahona, , Cristobal Otero Ruiz-Tagle, C...",We study a regulation in Chile that mandates w...,Econometrica,2023.0,https://business.columbia.edu/faculty/research...
4,"Brandon Freiberg, and Sandra Matz . “Founder p...","Brandon Freiberg, Brandon Freiberg, and Sandra...",Technology startups play an essential role in ...,Psychological and Cognitive Sciences,2023.0,https://business.columbia.edu/faculty/research...
5,"Cowgill, Bo , Jorge Guzman , and Dany Bahar . ...","Bo Cowgill, Bo Cowgill, , Jorge Guzman, Jorge ...",This paper analyzes the entire business regist...,American Economic Association Papers and Proce...,2023.0,https://business.columbia.edu/faculty/research...
6,"Peters, Heinrich , Sandra Matz , and Moran Cer...","Heinrich Peters, , Sandra Matz, Sandra Matz, ,...",NaN,NaN,2023.0,https://business.columbia.edu/faculty/research...
7,"Fradkin, Andrey and David Holtz . “Do Incentiv...","Andrey Fradkin, and David Holtz, David Holtz",Many online reputation systems operate by aski...,Marketing Science,2023.0,https://business.columbia.edu/faculty/research...
8,"Agarwal, Sumit , Gene Amromin , Souphala Choms...","Sumit Agarwal, , Gene Amromin, , Souphala Chom...",NaN,The Review of Economic Studies,2023.0,https://business.columbia.edu/faculty/research...
9,"Madan, Shilpa , Gita Johar , Jonah Berger , Pi...","Shilpa Madan, , Gita Johar, Gita Johar, , Jona...","Over the past several decades, scholars have h...",Marketing Letters,2023.0,https://business.columbia.edu/faculty/research...
